In [5]:
print(s3_access_key)

None


In [1]:
import requests
import os
import json
from minio import Minio
import io
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = os.environ.get("TWITTER_BEARER_TOKEN")

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FilteredStreamPython"
    return r


def get_rules():
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    return response.json()


def delete_all_rules(rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print(json.dumps(response.json()))


def set_rules(delete):
    # You can adjust the rules if needed
    sample_rules = [
        {"value": "dog has:images", "tag": "dog pictures"},
        {"value": "cat has:images -grumpy", "tag": "cat pictures"},
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))


def get_stream(set):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream", auth=bearer_oauth, stream=True,
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    i = 0
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            formatted = json.dumps(json_response, indent=4, sort_keys=True)
            print(formatted)
            s3_client.put_object(bucket, f"test_tweets/{i}.json", io.BytesIO(bytes(formatted, "utf-8")), len(formatted))



In [2]:
s3_access_key = os.environ.get("S3_ACCESS_KEY")
s3_secret_key = os.environ.get("S3_SECRET_KEY")
bucket = "twitter-data-123456"
s3_client = Minio("storage.googleapis.com", access_key=s3_access_key, secret_key=s3_secret_key)

In [3]:
rules = get_rules()
delete = delete_all_rules(rules)
set = set_rules(delete)

{"data": [{"id": "1486190908278247426", "value": "cat has:images -grumpy", "tag": "cat pictures"}, {"id": "1486190908278247427", "value": "dog has:images", "tag": "dog pictures"}], "meta": {"sent": "2022-01-26T04:28:21.533Z", "result_count": 2}}
{"meta": {"sent": "2022-01-26T04:28:22.759Z", "summary": {"deleted": 2, "not_deleted": 0}}}
{"data": [{"value": "cat has:images -grumpy", "tag": "cat pictures", "id": "1486194281715056640"}, {"value": "dog has:images", "tag": "dog pictures", "id": "1486194281715056641"}], "meta": {"sent": "2022-01-26T04:28:24.989Z", "summary": {"created": 2, "not_created": 0, "valid": 2, "invalid": 0}}}


In [4]:
get_stream(set)

200
{
    "data": {
        "id": "1486194257320841217",
        "text": "\u306a\u3093\u3067\u7121\u96e3\u3067\u96d1\u306a\u8fd4\u7b54\u3092\n\u5dee\u3057\u8fbc\u3080\u306a\u3069\u3059\u308b\n(\u30c9\u30fc\u30d9\u30eb\u306f\u5c02\u9580\u5916\u306a\u3093\u3060\u3088\u7533\u3057\u8a33\u306a\u3044\u3051\u3069\u3055\u3042) https://t.co/lR4eg7RheH"
    },
    "matching_rules": [
        {
            "id": "1486194281715056640",
            "tag": "cat pictures"
        }
    ]
}


S3Error: S3 operation failed; code: AccessDenied, message: Access denied., resource: None, request_id: None, host_id: None